In [2]:

import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.dates as mdates
from matplotlib import pyplot as plt
from uszipcode import SearchEngine
from IPython.core.display import HTML, display
from matplotlib import cm
import matplotlib.dates as mdates
from datetime import datetime
%matplotlib inline
plt.style.use('ggplot')
pd.set_option('display.max_columns', 500)
pd.options.display.float_format = '{:,.2f}'.format
display(HTML("<style>.container { width:95% !important; }</style>"))

In [3]:
ratings = pd.read_csv('ml-100k/u.data', header=None, sep='\t', encoding='latin-1', names=["user_id", "movie_id", "rating", "timestamp"])
movies = pd.read_csv('ml-100k/u.item', header=None, sep='|', encoding='latin-1', names=["movie_id","title","release_date","video_release_date","imdb_url","unknown","action","adventure","animation","children","comedy","crime","documentary","drama","fantasy","film-noir","horror","musical","mystery","romance","sci-fi","thriller","war","western"])
users = pd.read_csv('ml-100k/u.user', header=None, sep='|', encoding='latin-1', names=["user_id", "age", "gender","occupation","zip_code"])

In [4]:
ratings.head()
ratings['timestamp_dt'] = pd.to_datetime(ratings['timestamp'], unit='s')
ratings.head()

,user_id,movie_id,rating,timestamp,timestamp_dt
0,196,242,3,881250949,1997-12-04 15:55:49
1,186,302,3,891717742,1998-04-04 19:22:22
2,22,377,1,878887116,1997-11-07 07:18:36
3,244,51,2,880606923,1997-11-27 05:02:03
4,166,346,1,886397596,1998-02-02 05:33:16


In [5]:
movies.info()
movies.drop(columns=['video_release_date', 'imdb_url'], inplace=True)
movies.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1682 entries, 0 to 1681
Data columns (total 24 columns):
movie_id              1682 non-null int64
title                 1682 non-null object
release_date          1681 non-null object
video_release_date    0 non-null float64
imdb_url              1679 non-null object
unknown               1682 non-null int64
action                1682 non-null int64
adventure             1682 non-null int64
animation             1682 non-null int64
children              1682 non-null int64
comedy                1682 non-null int64
crime                 1682 non-null int64
documentary           1682 non-null int64
drama                 1682 non-null int64
fantasy               1682 non-null int64
film-noir             1682 non-null int64
horror                1682 non-null int64
musical               1682 non-null int64
mystery               1682 non-null int64
romance               1682 non-null int64
sci-fi                1682 non-null int64
thriller 

,movie_id,title,release_date,unknown,action,adventure,animation,children,comedy,crime,documentary,drama,fantasy,film-noir,horror,musical,mystery,romance,sci-fi,thriller,war,western
0,1,Toy Story (1995),01-Jan-1995,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0


In [6]:
movies.iloc[:, 3:].sum()

unknown          2
action         251
adventure      135
animation       42
children       122
comedy         505
crime          109
documentary     50
drama          725
fantasy         22
film-noir       24
horror          92
musical         56
mystery         61
romance        247
sci-fi         101
thriller       251
war             71
western         27
dtype: int64

In [7]:
movies.describe().transpose()



,count,mean,std,min,25%,50%,75%,max
movie_id,"1,682.00",841.50,485.70,1.00,421.25,841.50,"1,261.75","1,682.00"
unknown,"1,682.00",0.00,0.03,0.00,0.00,0.00,0.00,1.00
action,"1,682.00",0.15,0.36,0.00,0.00,0.00,0.00,1.00
adventure,"1,682.00",0.08,0.27,0.00,0.00,0.00,0.00,1.00
animation,"1,682.00",0.02,0.16,0.00,0.00,0.00,0.00,1.00
children,"1,682.00",0.07,0.26,0.00,0.00,0.00,0.00,1.00
comedy,"1,682.00",0.30,0.46,0.00,0.00,0.00,1.00,1.00
crime,"1,682.00",0.06,0.25,0.00,0.00,0.00,0.00,1.00
documentary,"1,682.00",0.03,0.17,0.00,0.00,0.00,0.00,1.00
drama,"1,682.00",0.43,0.50,0.00,0.00,0.00,1.00,1.00


In [8]:
search = SearchEngine(simple_zipcode=True)

In [9]:
users['lat'] = users['zip_code'].apply(lambda x: search.by_zipcode(x).lat)
users['lon'] = users['zip_code'].apply(lambda x: search.by_zipcode(x).lng)
users['income'] = users['zip_code'].apply(lambda x: search.by_zipcode(x).median_household_income)

In [10]:
users.head()

,user_id,age,gender,occupation,zip_code,lat,lon,income
0,1,24,M,technician,85711,32.21,-110.89,"32,557.00"
1,2,53,F,other,94043,37.41,-122.09,"97,552.00"
2,3,23,M,writer,32067,nan,nan,nan
3,4,24,M,technician,43537,41.57,-83.68,"60,078.00"
4,5,33,F,other,15213,40.44,-79.96,"21,928.00"


In [11]:
from keplergl import KeplerGl

users_map = KeplerGl(height=800, data = {'data': users})
users_map

User Guide: https://github.com/keplergl/kepler.gl/blob/master/docs/keplergl-jupyter/user-guide.md


KeplerGl(data={'data':      user_id  age gender     occupation zip_code   lat     lon     income
0          1 …